# Get Besancon samples for TESS fields

The purpose of this code is to pull down galaxy model data fom the Besancon model for fields of view that equate to TESS pointings. The point of this is that we are going to use these to simulate what we think TESS will find. First we need to get estimate of the numbers of stars that they can target.

In [1]:
from __future__ import division, print_function
import numpy as np
import pandas as pd
import astroquery
import matplotlib.pyplot as plt

%matplotlib inline

First we want to think about how to get coordinates for all the different TESS pointings. 
TESS has 26 observing sectors [see http://tess.gsfc.nasa.gov/science.html](http://tess.gsfc.nasa.gov/science.html) and [http://arxiv.org/pdf/1406.0151v3.pdf](http://arxiv.org/pdf/1406.0151v3.pdf). 

These sectors are each 4 detectors of 24x24 degrees. We should use ecliptic coordinates.
The first row of CCDS has latitude of +/- 18 degrees. There are thirteen CCDs so the center longitudes must be np.linspace(0,360,14) (because 0==360).

So to get the first 26 CCD centers we can do 

In [2]:
lon = np.repeat(np.linspace(0,360,14)[:-1],2)
lat = np.tile([18,-18],13)
centers_row1 = np.array([lon,lat]) # long/lat in degrees
areas_row1 = np.repeat(24*24,26) # area in sq deg = 24*24

Next we want to think about the next row of centers. The latitudes are very simple here: +/- 42 degrees. 


In [3]:
lat = np.tile([42,-42],13)
centers_row2 = np.array([lon,lat]) # long/lat in degrees

There are again 13 rows but this time we want to think carefully about the areas because there are overlaps.

The area of the spherical zone can be calculated using the area of a spherical cap as 
A = 2*pi*(1- cos( 90-lat ) )

So... area of the how segment is

In [4]:
A = (2*np.pi*(1 - np.cos(np.radians(90 - (42-12)))) 
     - 2*np.pi*(1 - np.cos(np.radians(90 - (42+12) )  )))

And therefore each individual fov has effective area of 

In [5]:
areas_row2 = np.repeat((A * (180/np.pi)**2)/13,26)

Ok, do the same for the next row. Lat centers at 42+24 degrees = 66

In [6]:
lat = np.tile([66,-66],13)
centers_row3 = np.array([lon,lat]) # long/lat in degrees

A = (2*np.pi*(1 - np.cos(np.radians(90 - (66-12)))) 
     - 2*np.pi*(1 - np.cos(np.radians(90 - (66+12) )  )))
areas_row3 = np.repeat((A * (180/np.pi)**2)/13,26)

Finally, we only need one for each pole.

In [7]:
centers_row4 = np.array([[0,0],[90,-90]]) # long/lat in degrees

In [8]:
areas_row4 = np.repeat(24*24,2)

This gives us, in total, 80 pointings to consider (26+26+26+2).

### Now to query besancon

We want to import the astroquery module to get the Besancon model
we also want a foolproof way to convert from ecliptic to galactic coordiantes

In [9]:
from astroquery.besancon import Besancon
import ephem

In [10]:
maglim = {'U':(-99, 99), 'B':(-99, 99), 'V':(10, 18), 'R':(-99, 99),
          'I':(4, 13), 'J':(-99, 99), 'H':(-99, 99), 'K':(-99, 99),
          'L':(-99, 99)}

absmaglim = (-7, 20)



We only want stars in the mag range of i = [4:13] because this is what TESS will target per their paper.

In [15]:
for center in np.r_[centers_row1.T,centers_row2.T,centers_row3.T,centers_row4.T]:
    thisEcl = center
    ecl_obj = ephem.Ecliptic(*np.radians(thisEcl))
    gal_obj = ephem.Galactic(ecl_obj)

    model = Besancon.query(glon=np.float(gal_obj.lon), glat=np.float(gal_obj.lat),
                           email='tom@tombarclay.com',
                           area=1.0,
                           mag_limits=maglim,
                           absmag_limits=absmaglim,
                           verbose=True)

    model.write('besmod_{}_{}.csv'.format(*thisEcl), format='ascii', delimiter=',')

Loading request from Besancon server ...
File is 1430889508.204953.resu and can be aquired from ftp://sasftp.obs-besancon.fr/modele/modele2003//1430889508.204953.resu
Awaiting Besancon file...


KeyboardInterrupt: 

In [15]:
centers = np.r_[centers_row1.T,centers_row2.T,centers_row3.T,centers_row4.T]
areas = np.r_[areas_row1,areas_row2,areas_row3,areas_row4]


np.savetxt('bess_reas.txt', np.c_[centers,areas])

array([576, 576, 576, 576, 576, 576, 576, 576, 576, 576, 576, 576, 576,
       576, 576, 576, 576, 576, 576, 576, 576, 576, 576, 576, 576, 576])